## Q1. Running Elastic

`build_hash = 42f05b9372a9a4a470db3b52817899b99a76ee73`

### Getting the data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '3d5fe55894e6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'NAILiom5Q--vaCLqfwtkaA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

/opt/homebrew/Caskroom/miniforge/base/envs/llm-zoomcamp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 531.43it/s]


Which function do you use for adding your data to elastic?
- `index`

## Q3. Searching

In [22]:
query = "How do I execute a command in a running docker container?"

search_query_q3 = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "machine-learning-zoomcamp"
            #     }
            # }
        }
    }
}

response = es_client.search(index=index_name, body=search_query_q3)

In [12]:
import pandas as pd

In [16]:
response_df = pd.DataFrame(response['hits']['hits'])
response_df

,_index,_id,_score,_source
0,course-questions,GKlqRpABHqqBKPdRT1af,84.050095,{'text': 'Launch the container image in intera...
1,course-questions,JqlqRpABHqqBKPdRS1Tx,75.541280,{'text': 'In case running pgcli locally cause...
2,course-questions,R6lqRpABHqqBKPdRUVd7,72.085180,{'text': 'If you are trying to run Flask gunic...
3,course-questions,N6lqRpABHqqBKPdRT1bP,51.046280,{'text': 'You can copy files from your local m...
4,course-questions,OKlqRpABHqqBKPdRT1bR,49.938507,{'text': 'You can copy files from your local m...


What's the score for the top ranking result?
- `84.05`

## Q4. Filtering

In [32]:
search_query_q4 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query_q4)

In [33]:
questions_from_response = [(r['_source']['question'], r['_score']) for r in response['hits']['hits']]

In [34]:
questions_from_response

[('How do I debug a docker container?', 84.050095),
 ('How do I copy files from my local machine to docker container?', 51.04628),
 ('How do I copy files from a different folder into docker container’s working directory?',
  49.938507)]

In [40]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'GKlqRpABHqqBKPdRT1af',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'N6lqRpABHqqBKPdRT1bP',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

Return 3 results. What's the 3rd question returned by the search engine?
- 'How do I copy files from a different folder into docker container’s working directory?

## Q5. Building a prompt

In [35]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [36]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [53]:
context = '\n\n'.join(context_template.format(question=doc['_source']['question'],
                                              text=doc['_source']['text'])
                      for doc in response['hits']['hits'])
prompt = prompt_template.format(question=query, context=context)

In [54]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [55]:
len(prompt)

1462

What's the length of the resulting prompt? (use the len function)
- `1462`

## Q6. Tokens

In [59]:
import tiktoken

In [60]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [62]:
tokens = encoding.encode(prompt)

In [65]:
tokens

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 8470,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,
 9282,
 3901,
 32,
 2

In [78]:
encoding.decode([4165,14029])

' course teaching'

In [ ]:
encoding

In [63]:
len(tokens)

322

Use the encode function. How many tokens does our prompt have?
- `322`

## Generate the answer

In [81]:
import ollama

In [82]:
response = ollama.chat(model='mistral:instruct', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

In [87]:
print(response['message']['content'])

 To execute a command in a running Docker container, follow these steps:

1. First, find the container-id using `docker ps`.
2. Then, execute the command in the specific container with the following command:

```bash
docker exec -it <container-id> bash
```

This will open a new terminal inside your running Docker container and allow you to execute commands as if you were directly on the container.
